## Notebook Principal

Contém os testes com a árvore trie, além de exemplos de compressão e decompressão

In [1]:
from trie import RadixTree
from lzw import *
from functions import *
import pandas as pd
import os
import time

# Testes com Radix Tree

In [ ]:
tree = RadixTree()
tree.insert_many(["word", "word1", "word2", "word3", "word12", "abc", "a", "bc"])
print(tree)
tree.print_tree()

{'word': 0, 'word1': 1, 'word12': 4, 'word2': 2, 'word3': 3, 'a': 6, 'abc': 5, 'bc': 7}
--- word:  	(0)
------ 1:  	(1)
--------- 2:  	(4)
------ 2:  	(2)
------ 3:  	(3)
--- a:  	(6)
------ bc:  	(5)
--- bc:  	(7)


# Função para escrever os códigos criados num arquivo de texto

# Script de testes

1. Compressão:
    * Codificação de textos
    * Criação de lista de códigos
    * Compressão dos códigos em binário
    * Comparação do tamanho dos textos com os códigos binários
    * Taxa de compressão
    * Tempo de compressão

2. Decompressão
    * Decompressão dos códigos binários
    * Decodificação dos códigos
    * Comparação do texto decodificado com o arquivo original
    * Tempo de decompressão

In [ ]:
def compress_data(path, files):
    df = pd.DataFrame(
        columns=[
            "Arquivo",
            "Formato",
            "Tamanho original (bits)",
            "Tamanho comprimido (bits)",
            "Tamanho Dicionário",
            "Qtd de bits por código",
            "Taxa de compressão",
            "Tempo de compressão (ms)",
            "Tempo Relativo (ms/kB)",
        ]
    )
    for file_name in files:
        f = path + "/" + file_name
        with open(f, "r", encoding="utf-8") as file:
            s = file.read()
            file_size = len(s)
            print("-" * 50)
            print(f"Arquivo de entrada: {file_name}")
            print(f"Tamanho: {len(s)} bytes")

            for format in ["ascii", "binary"]:
                print("-" * 50)
                print(f"Encoding {format} text...")
                if format == "binary":
                    s = str_to_bin(s)

                # Repete 10 vezes
                for i in range(10):

                    # Codificação
                    start_time = time.time()
                    encoder = LZW_Encoder(by=format)
                    encoded = encoder.encode(s)
                    compressed, n_bits = compress(encoded)
                    end_time = time.time()

                    # Fim da codificação
                    # Dados para o Dataframe
                    dictionary = encoder.dictionary
                    dict_size = len(dictionary)
                    print(f"Tamanho do dicionário: {dict_size} strings\n")
                    code_time = end_time - start_time
                    rel_time = code_time / file_size
                    compressed_size = len(compressed)
                    compressed_rate = file_size / compressed_size
                    df.loc[len(df)] = [
                        file_name,  # Nome do arquivo
                        format,  # 'ascii' ou 'binary'
                        8 * file_size,  # Tamanho em bits
                        len(compressed),  # Tamanho em bits
                        dict_size,  # Número de strings
                        n_bits,  # Bits por código
                        8 * compressed_rate,  # Taxa de compressão
                        1000 * code_time,  # Tempo em ms
                        1000 * 1024 * rel_time,  # Tempo relativo em ms/kB
                    ]
                    print(f"Tempo de compressão: %.3f ms" % (1000 * code_time))
                    print(f"Tempo normalizado:   %.3f ms/kB" % (1000 * 1024 * rel_time))
                    print(f"Tamanho original:    %d bits" % (8 * file_size))
                    print(f"Tamanho comprimido:  %d bits" % (compressed_size))
                    print(f"Taxa de compressão:  %.3f" % (8 * compressed_rate))
    return df

In [ ]:
def main():
    """
    Exemplo de uso.
    """
    # Arquivos de entrada
    n_files = 5
    path = "tests"
    files = [f"ex ({i}).txt" for i in range(1, n_files + 1)]  # "ex (i).txt"

    # Criação do Dataframe
    df = compress_data(path, files)
    print(df)
    df.to_csv("out.csv", index=False)

In [ ]:
if __name__ == "__main__":
    main()

--------------------------------------------------
Arquivo de entrada: ex (1).txt
Tamanho: 18 bytes
--------------------------------------------------
Encoding ascii text...
Tamanho do dicionário: 271 strings

Tempo de compressão: 0.395 ms
Tempo normalizado:   22.488 ms/kB
Tamanho original:    144 bits
Tamanho comprimido:  144 bits
Taxa de compressão:  1.000
Tamanho do dicionário: 271 strings

Tempo de compressão: 0.317 ms
Tempo normalizado:   18.039 ms/kB
Tamanho original:    144 bits
Tamanho comprimido:  144 bits
Taxa de compressão:  1.000
Tamanho do dicionário: 271 strings

Tempo de compressão: 0.316 ms
Tempo normalizado:   17.999 ms/kB
Tamanho original:    144 bits
Tamanho comprimido:  144 bits
Taxa de compressão:  1.000
Tamanho do dicionário: 271 strings

Tempo de compressão: 0.292 ms
Tempo normalizado:   16.629 ms/kB
Tamanho original:    144 bits
Tamanho comprimido:  144 bits
Taxa de compressão:  1.000
Tamanho do dicionário: 271 strings

Tempo de compressão: 0.295 ms
Tempo normal